In [ ]:
# SEGMENTACIÓN MEDIANTE THRESHOLDING
# Read an image
import matplotlib.pyplot as plt
import cv2
img = cv2.imread('images/figuras.png')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb)
plt.title('Imagen original')
plt.show()

In [ ]:
# Convertimos la imagen a escala de grises
import numpy as np
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#gray_img = cv2.imread('figuras.png', 0)

plt.imshow(gray_img, cmap='gray')
plt.title('Imagen de grises')
plt.show()

In [ ]:
# Calculamos el histograma
hist = cv2.calcHist([gray_img], [0], None, [260], [0, 260]) # [0] son los canales, None la máscara

plt.plot(hist)
plt.title('Histograma')
plt.show()

# Otra forma de mostrar el histograma
# plt.hist(gray_img.ravel())
# plt.grid(True)
# plt.show()

In [ ]:
# Fijamos el umbral en base al histograma anterior
t = 50

# Extreaemos la máscara binaria
maxim = int(np.max(np.unique(gray_img)))
_, mask = cv2.threshold(gray_img, t, int(maxim), cv2.THRESH_BINARY)

# Otra formas de extraer la máscara
# mask = gray_img.copy()
# mask[mask>t]=255

# Visualizamos para corroborar
plt.imshow(mask, cmap='gray')
plt.title('Máscara t=' + str(t))
plt.show()

In [ ]:
# Fijamos el umbral con el método de OTSU
t, mask = cv2.threshold(gray_img,0,1,cv2.THRESH_BINARY+cv2.THRESH_OTSU) # 0 es por defecto y 1 es el valor máximo de la máscara
print(np.unique(mask))

# Visualizamos para corroborar que se obtiene el mismo resultado
plt.imshow(mask, cmap='gray')
plt.title('Máscara Otsu t=' + str(t))
plt.show()

In [ ]:
###############################################################################################################################

In [ ]:
# SEGMENTACIÓN MEDIANTE CONVERSIONES DE COLOR

# Imaginamos que queremos segmentar únicamente el círculo
img = cv2.imread('images/figuras.png')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img_rgb)
plt.title('Imagen original')
plt.show()

In [ ]:
# Obtenemos el canal rojo
R = img[:,:,2]

plt.imshow(R, cmap='gray')
plt.title('Canal rojo')
plt.show()

In [ ]:
# Observamos el histograma
hist = cv2.calcHist([R], [0], None, [260], [0, 260])

plt.plot(hist)
plt.title('Histograma')
plt.show()

In [ ]:
# Fijamos el umbral y extraemos la máscara
t=250
mask = R.copy()
mask[mask<250] = 0

plt.imshow(mask, cmap='gray')
plt.title('ROI segmentada')
plt.show()

In [ ]:
###############################################################################################################################

In [ ]:
# SEGMENTACIÓN SEMÁNTICA CON DETECCIÓN DE BORDES
img = cv2.imread('images/figuras.png')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img_rgb)
plt.title('Imagen original')
plt.show()

In [ ]:
# Aplicamos un filtro gaussiano para emborronar las altas frecuencias
img_gaus = cv2.GaussianBlur(img_rgb, (5,5), 0) # (5x5) es el tamaño del filtro y 0 es la desviación estándar
plt.imshow(img_gaus, cmap='gray')
plt.show()

In [ ]:
# Detección de bordes de Canny
canny = cv2.Canny(img_gaus, 255/3, 255) # minValue and maxValue. Canny recomiendo que minValue sea 3 veces mayor que maxValue
kernel = np.ones((2,2), 'uint8')
canny = cv2.dilate(canny, kernel, iterations=2)
canny = cv2.erode(canny, kernel, iterations=2)

plt.imshow(canny, cmap='gray')
plt.show()

In [ ]:
# Rellenar objetos (solo rellena si no están en contacto con el límite de la imagen)
from scipy.ndimage.morphology import binary_fill_holes as bfh
bordes = canny.copy()
filled_bordes = bfh(bordes)

plt.imshow(filled_bordes.astype('uint8'), cmap='gray')
plt.title('cleaned lumen mask')
plt.show()

In [ ]:
# Otra forma de rellenar los bordes (Algoritmo de expansión a partir de semillas)
h, w = canny.shape
ref = np.zeros((h+2, w+2), np.uint8)
# Se fija la semilla en la esquina superior izquierda (posición 0,0) para rellenar a partir de ahí
cv2.floodFill(canny, ref, (0,0), 255); 

# Se calcula el inverso del rellenado para obtener la máscara deseada
mask = cv2.bitwise_not(canny)

fig, ax = plt.subplots(1,2)
ax[0].imshow(canny, cmap='gray'), ax[0].set_title('rellenado')
ax[1].imshow(mask, cmap='gray'), ax[1].set_title('máscara')
plt.show()